In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
# Copy Files From External Drive To Local Disk

from data.file_utils import Files
import shutil
import os
import numpy as np

db_folder = "/Volumes/DIY_NVME/Bio-informatic project/Exp_20240213_N2_JC01/"
# raw_ds_folder = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_PARTIAL2/"
raw_ds_folder = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_RAND/"

file_iter = Files(db_folder)
print(len(file_iter))

video_length = 1000
sample_start_list = range(8000, 58000, 3700)
rnd_fids = np.random.choice(len(file_iter), size=1000, replace=False)

# for i in rnd_fids:
#     src = file_iter.seek(i)
#     filename = file_iter.get_filename()
#     dst = raw_ds_folder + filename
#     shutil.copyfile(src, dst)

# for vid_start in sample_start_list:
#     file_iter.seek(vid_start-1)
#     ret = input(f"Move {video_length} files, strating after {file_iter.get_filename()}? [y/n]")
#     if ret.upper() != 'Y':
#         break
#     for a in range(video_length):
#         src = file_iter.__next__()
#         filename = file_iter.get_filename()
#         dst = raw_ds_folder + filename
#         # os.symlink(src, dst)
#         shutil.copyfile(src, dst)





In [ ]:
cv.destroyAllWindows()

In [ ]:
def image_transform(image: np.ndarray):
    # remove gaussian noise
    image = cv.GaussianBlur(image, (3, 3), 0)

    # quantize image
    N = 50
    image = np.round(image * (N / 255), decimals=0) * (255 / N)

    return image.astype(np.uint8)

In [1]:
from dataclasses import dataclass
# class to hold the db paths of our pipeline 
@dataclass(frozen=True)
class PATHS:
    raw_db:str
    cropped_db:str
    dlc_db:str


In [ ]:
# define the paths

db_paths =  PATHS(raw_db="/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_02_PARTIAL/",
            cropped_db="/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_02_EXTRACTED/",
            dlc_db="/Users/guycohen/Desktop/Test-Dataset-2024-02-26/labeled-data/output/",
            )
db_paths2 =  PATHS(raw_db="/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_RAND/",
            cropped_db="/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_EXTRACTED2/",
            dlc_db="/Users/guycohen/Desktop/Test-Dataset-2024-02-26/labeled-data/output/",
            )

In [2]:
db_paths3 =  PATHS(raw_db="/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_PARTIAL",
                   cropped_db="/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_EXTRACTED3000/sample0",
                   dlc_db="asdasdasd",
                  )

In [ ]:
from data.frame_reader import FrameReader
from data.video_extractor import VideoExtractor
from data.file_utils import Files




files = [f for f in Files(db_paths3.raw_db)][:3000]

reader = FrameReader(db_paths3.raw_db, files)
extractor = VideoExtractor(reader, num_workers=4)

In [ ]:
extractor.generate_all_videos((400,400), "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/EXP_01_EXTRACTED3000/sample{}")

In [ ]:
# Initialize frame reader and extractor

from data.frame_reader import FrameReader
from data.video_extractor import VideoExtractor


reader = FrameReader.create_from_directory(db_paths2.raw_db)
extractor = VideoExtractor(reader, num_workers=4)

In [ ]:
sample_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/samples"

extractor.background()

extractor.create_samples(1000, (400,400), sample_path)

In [ ]:
reader = FrameReader.create_from_directory(sample_path)
for f in reader:
    if reader.frame_shape != (400,400):
        print("ERROR")

In [ ]:
from dataset.raw_dataset import *
from data.file_utils import pickle_load_object, pickle_save_object

background = extractor.background()

h, w = reader.frame_shape
exp_meta = ExperimentMeta(60, (w, h), 133.33333, background)
experiment = ExperimentDataset.create_from_frame_reader(reader, exp_meta)

In [ ]:
from data.file_utils import folder_convert_images, rename_raw_file_names, Files
import os

# sample_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/samples/"

# file_iter = Files(sample_path, 'png')

# for file in file_iter:
#     # print(file[:-len(file_iter.get_filename())])
#     new_name = file_iter.get_filename().split('-')[-1]
#     dst = sample_path + new_name
#     os.rename(file, dst)



In [ ]:
# Copy the extracted images to the dlc labeling folder

from data.file_utils import Files
import shutil

every_x_samples = 30
count = 0

directories = Files(db_paths.cropped_db, scan_dirs=True)
for dir in directories:
    file_iter = Files(dir, '.png')
    count += len(file_iter)
    for i in range(0, len(file_iter), every_x_samples):
        file = file_iter.seek(i)
        shutil.copyfile(file, db_paths.dlc_db+file_iter.get_filename())

print(count)

In [ ]:
from view_controller import ViewController

view = ViewController(FrameReader.create_from_directory("sample_all_0"))

while True:
    #view.move_position(5, 5)
    view.visualize_world()
    cv.imshow("micro", view.camera_view())
    cv.waitKey(0)

    if view.progress() == False:
        break
    
cv.destroyAllWindows()

In [ ]:
raise Exception("Finished")

In [ ]:
# TODO: IMPROVE
def find_boxes(image: np.ndarray) -> np.ndarray:
    # increase contrast of original image
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # remove gaussian noise
    smoothed = cv.GaussianBlur(image, (3, 3), 0)

    # apply threshold to image to convert it into black & white
    _, mask = cv.threshold(smoothed, 160, 255, cv.THRESH_BINARY_INV)

    # do some morphological magic to clean up noise from the mask
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)

    # dilate to increase all object sizes in the mask
    kernel = np.ones((3, 3), np.uint8)
    mask = cv.dilate(mask, kernel, iterations=5)

    image[mask == 0] = 255
    cv.imshow("masked", image)

    # find contours
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bbox_list = []
    for c in contours:
        area = cv.contourArea(c)

        if area < 500 or area > 4000:
            continue

        box = cv.boundingRect(c)
        bbox_list.append(box)

    # Turn our bboxes into 2d ndarray
    bboxes = np.asanyarray(bbox_list)
    return bboxes

In [ ]:
from view_controller import VideoStream
from data.legacy_box_extractor import BoxExtractor

video = VideoStream("worms.avi", padding_size=(251, 251))
extractor = BoxExtractor(find_boxes, slice_size=251, object_size=150)

for frame in video:
    if cv.waitKey(1) & 0xFF == ord("q"):
        break

    head_coords, rois = extractor.run(frame)

    # Draw bboxes
    for box in head_coords:
        x, y, w, h = box
        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 3)

    cv.imshow("BBOXES", frame)

In [3]:
import ultralytics
from data.file_utils import Files 
from data.frame_reader import FrameReader
import cv2 as cv

# files = Files(db_paths3.cropped_db)
reader = FrameReader.create_from_directory(db_paths3.cropped_db, cv.IMREAD_COLOR)

model = ultralytics.YOLO('/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Bio-Proj/yolov8m-trained.pt')




# file = files.__next__()



In [ ]:
imgs = [reader[i] for i in range(0,1000,60)]
pred = model.predict(imgs, show_labels=False, line_width=3)

In [ ]:
pred[0].boxes

In [4]:
import math
from view_controller import ViewController
from data.file_utils import Files




files = [f for f in Files(db_paths3.raw_db)][:3000]
full_reader = FrameReader(db_paths3.raw_db, files, cv.IMREAD_COLOR)

sim = ViewController(full_reader, (401,401), (20,20), (1400, 1200), padding_value=[255,255,255])

fps = 60
micro_view_time = 200
tracking_time = 50
moving_time = 25

frame_time = 1000 / fps
micro_view_frames = math.ceil(micro_view_time / frame_time)
tracking_frames = math.ceil(tracking_time / frame_time)
moving_frames = math.ceil(moving_time / frame_time)

In [ ]:


sim.visualize_world()


In [ ]:
import os
import threading
from data.tqdm_utils import TqdmQueue

class ImageSaver:
    def __init__(self) -> None:
        self.progress_queue = TqdmQueue(desc="Saving videos", unit="vid")
        self.worker_thread = threading.Thread(target=self._video_saver_worker, args=(self.progress_queue,))
        
        pass
    
    def start(self):
        self.worker_thread.start()

    def _video_saver_worker(self, image_queue: TqdmQueue):
        """
        Worker method that processes video saving tasks from the video_params queue.

        Args:
            video_params (queue.Queue): The queue containing video saving tasks.

        Returns:
            None
        """
        while True:
            task = image_queue.get()

            # exit if signaled
            if task is None:
                break

            image, path = task
            cv.imwrite(image, path)
            image_queue.task_done()

    def save_image(self, image:np.ndarray, path:str):
        self.progress_queue.put((image, path))
    
    def close(self):
        self.progress_queue.join()  # wait for queue to empty
        self.progress_queue.put(None)  # put stop signal into queue
        self.worker_thread.join()  # wait for worker thread to finish
        

def create_boundry(image:np.ndarray, boundry_size:int, color:int):
    image[:boundry_size, :] = color
    image[-boundry_size:, :] = color
    image[:, :boundry_size] = color
    image[:, -boundry_size:] = color
    return image

def simulate(sim:ViewController, ouput_folder:str):
    mic_path = ouput_folder + "micro/"
    err_path = ouput_folder + "errors/"
    os.makedirs(mic_path)
    os.makedirs(err_path)

    for i, _ in enumerate(sim):
        micro = sim.micro_view()

        if i%(moving_frames+micro_view_frames):
            cam_view = sim.camera_view()
            pred = model.predict(cam_view, conf=0.1)
            dx, dy = (0,0)
            if len(pred) > 0:
                pred_box = pred[0].boxes.xyxy[0]
                pred_center = (((pred_box[0]+pred_box[2])/2).to(int).item(), ((pred_box[1]+pred_box[3])/2).to(int).item())
                dx, dy = (pred_center[0]-200, pred_center[1]-200)  # Takes 'track_frames' frames, can't do anything meanwhile
            else:
                print(f"ERROR:: No Head detected in frame {i}")
                cv.imwrite(err_path+"NoPred_{i}_cam.png", sim.camera_view())
                cv.imwrite(err_path+"NoPred_{i}_micro.png", sim.micro_view())
        
        #Change pos - end of movement
        if (i-tracking_frames) % (moving_frames+micro_view_frames):
            sim.move_position(dx, dy)
        
        # All movement phase
        if (i % (moving_frames+micro_view_frames)) <= moving_frames:
            file_path = ouput_folder + f"frame_{i}.png"
            cv.imwrite(mic_path, create_boundry(micro,2,0))
            continue

        file_path = ouput_folder + f"frame_{i}.png"
        cv.imwrite(mic_path, micro)

In [7]:
import matplotlib.pyplot as plt
import torch

from image_utils import show_image
plt.ioff()
%matplotlib qt

ouput_folder = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Bio Project/Data/MicView_Test/"

sim.reset()

for i, frame in enumerate(sim):
    NoPred = False
    if i%(moving_frames+micro_view_frames):
        cam_view = sim.camera_view()
        pred = model.predict(cam_view, conf=0.1)
        dx, dy = (0,0)
        if len(pred) > 0:
            pred_box = pred[0].boxes.xyxy[0]
            pred_center = (((pred_box[0]+pred_box[2])/2).to(int).item(), ((pred_box[1]+pred_box[3])/2).to(int).item())
            dx, dy = (pred_center[0]-200, pred_center[1]-200)  # Takes 'track_frames' frames, can't do anything meanwhile
        else:
            print(f"ERROR:: No Head detected in frame {i}")
            cv.imwrite(ouput_folder+"errors/cam_{i}.png", sim.camera_view())
            cv.imwrite(ouput_folder+"errors/mic_{i}.png", sim.micro_view())
    
    #Change pos - end of movement
    if (i-tracking_frames) % (moving_frames+micro_view_frames):
        sim.move_position(dx, dy)
    # All movement phase
    if (i % (moving_frames+micro_view_frames)) <= moving_frames:
        file_path = ouput_folder + f"frame_{i}.png"
        micro = sim.micro_view()
        micro[:2, :] = 0
        micro[-2:, :] = 0
        micro[:, :2] = 0
        micro[:, -2:] = 0
        cv.imwrite(file_path, micro)
        continue

    file_path = ouput_folder + f"frame_{i}.png"
    cv.imwrite(file_path, sim.micro_view())



0: 416x416 1 Head, 630.7ms
Speed: 14.7ms preprocess, 630.7ms inference, 0.7ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 Head, 771.5ms
Speed: 6.2ms preprocess, 771.5ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 Head, 800.5ms
Speed: 7.2ms preprocess, 800.5ms inference, 4.1ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 Head, 737.6ms
Speed: 4.2ms preprocess, 737.6ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 Head, 643.7ms
Speed: 8.8ms preprocess, 643.7ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 Head, 591.8ms
Speed: 1.9ms preprocess, 591.8ms inference, 2.9ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 Head, 614.0ms
Speed: 1.6ms preprocess, 614.0ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)

0: 416x416 1 Head, 625.5ms
Speed: 3.6ms preprocess, 625.5ms inference, 0.8ms postprocess per image at shape (1, 3, 4

KeyboardInterrupt: 

In [5]:
import matplotlib.pyplot as plt
import torch
from image_utils import show_image
plt.ioff()
%matplotlib qt

ouput_folder = ""

sim.reset()
while True:
    # Here we assume that the worm is kinda centered on the screen
    # sim.visualize_world()
    # plt.waitforbuttonpress()
    frame_num = sim._idx
    show_image(sim.micro_view())  # Takes 'micro_frames' frames, can do other stuff meanwhile
    sim.progress(micro_view_frames - tracking_frames)

    cam_view = sim.camera_view()    
    pred_box = model.predict(cam_view)[0].boxes.xyxy[0]
    pred_center = (((pred_box[0]+pred_box[2])/2).to(int).item(), ((pred_box[1]+pred_box[3])/2).to(int).item())
    print(pred_center)

    dx, dy = (pred_center[0]-200, pred_center[1]-200)  # Takes 'track_frames' frames, can't do anything meanwhile
    sim.progress(tracking_frames)  # micro and tracking finished

    sim.progress(moving_frames)  # moving takes 'move_frames' frames, can do other stuff meanwhile
    sim.move_position(dx, dy)

True

0: 416x416 1 Head, 607.1ms
Speed: 2.7ms preprocess, 607.1ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 416)
(145, 217)
True

0: 416x416 1 Head, 561.3ms
Speed: 1.7ms preprocess, 561.3ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 416)
(198, 201)
True

0: 416x416 1 Head, 553.6ms
Speed: 1.7ms preprocess, 553.6ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 416)
(199, 202)
True

0: 416x416 1 Head, 558.2ms
Speed: 1.9ms preprocess, 558.2ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 416)
(200, 201)
True

0: 416x416 1 Head, 511.3ms
Speed: 2.6ms preprocess, 511.3ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 416)
(199, 201)
True

0: 416x416 1 Head, 547.3ms
Speed: 2.0ms preprocess, 547.3ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)
(196, 201)
True

0: 416x416 1 Head, 559.3ms
Speed: 3.4ms preprocess, 559.3ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 416)
(200, 201)
True



KeyboardInterrupt: 